In [7]:
'''STREAMLIT APP'''
import streamlit as st
import base64
from PIL import Image
import time
import PyPDF2
import camelot
import pandas as pd
import numpy as np
import re
from sorted_months_weekdays import Month_Sorted_Month
from ipynb.fs.full.Get_BankStatement_Data import preprocess,get_preprocessing_data

In [12]:

path='files/output.pdf'

pd.set_option('display.max_info_columns',300)

# image = Image.open('files/image.png')
# st.image(image, width=400)


def write_file(path):
    with open(path, "wb") as f:
        f.write(uploaded_file.getbuffer())


try:
    uploaded_file = st.file_uploader("Choose an document...", type=("pdf"))
    submit = st.button('Submit')
    if submit:
        if uploaded_file is not None:
            write_file(path)
            base64_pdf = base64.b64encode(uploaded_file.getvalue()).decode('utf-8')
            pdf_display = f'<embed src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf">'
            st.markdown(pdf_display, unsafe_allow_html=True)



    st.subheader('Password:')
    password = st.text_input('Optional->Enter password if required', type='password')
    if password == '':
        password = None


    st.subheader('Select your Bank')
    bankname = st.selectbox('',
                            ('Select..','ICICI', 'AXIS','SBI','STANDARD CHARTERED','HDFC','PNB','Central Bank Of India','IDBI'))



    if bankname!='Select..':
        @st.cache
        def get_final_data(path,bankname,password):
            final,stop_index=get_preprocessing_data(path,bankname,password)
            credit_transactions, monthly_transaction_history, loan_history_monthly, monthly_transaction_history_details, rent_transactions, \
            trial, loan_history, credit_repeated_history, debit_repeated_history=preprocess(final,stop_index)

            return credit_transactions, monthly_transaction_history, loan_history_monthly, monthly_transaction_history_details, rent_transactions, \
           trial, loan_history, credit_repeated_history, debit_repeated_history


        credit_transactions, monthly_transaction_history, loan_history_monthly, monthly_transaction_history_details, rent_transactions, \
        final, loan_history, credit_repeated_history, debit_repeated_history = get_final_data(path, bankname, password)


        st.subheader('Type of Details')
        option = st.selectbox('', ('Select..','Loan History','Monthly Loan History', 'Detailed Statement','Monthly Transaction History',
                                   'Monthly Transaction Summary','Credit Transactions','Rent Transactions','Credit Repeated Transactions',
                                   'Debit Repeated Transactions'))
        # submit_option = st.button('Submit Option')
        # if submit_option:

        st.write('You selected:', option)

        if option=='Select..':
            st.write('Please Choose an Option')
        if option=='Loan History':
            st.dataframe(loan_history)
        if option=='Detailed Statement':
            st.dataframe(final)
        if option =='Monthly Transaction Summary':
            st.dataframe(monthly_transaction_history_details)
        if option=='Credit Transactions':
            st.dataframe(credit_transactions)
        if option=='Rent Transactions':
            st.dataframe(rent_transactions)
        if option == 'Monthly Transaction History':
            month = st.selectbox('', tuple(monthly_transaction_history.keys()))
            submit_month = st.button('Submit Month')
            if submit_month:
                st.dataframe(monthly_transaction_history[month])
        if option== 'Monthly Loan History':
            month = st.selectbox('', tuple(loan_history_monthly.keys()))
            submit_month = st.button('Submit Month')
            if submit_month:
                st.dataframe(loan_history_monthly[month])
        if option=='Credit Repeated Transactions':
            st.dataframe(credit_repeated_history)
        if option=='Debit Repeated Transactions':
            st.dataframe(debit_repeated_history)
except TypeError:
    st.write('No PDF uploaded')
